<a href="https://colab.research.google.com/github/gabrielpelicolla/trabalhos_estudo_PLN/blob/main/Lista5PLN_GabrielPelicolla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação das Bibliotecas

##Instalar bibliotecas

In [2]:
!pip install --upgrade --force-reinstall numpy pandas scikit-learn gensim spacy matplotlib seaborn transformers torch --quiet
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

#Exemplo de Dataset

##Importação pandas

In [3]:
import pandas as pd


##Simulação de dataset

In [5]:
data = {
    "review": [
        "A câmera tem ótima qualidade, adorei!",
        "Bateria fraca, não dura nada.",
        "Design bonito e leve, mas o desempenho é ruim.",
        "Produto excelente, superou minhas expectativas.",
        "Não recomendo, veio com defeito."
    ],
    "label": [1, 0, 0, 1, 0]  # 1 = positivo, 0 = negativo
}

df = pd.DataFrame(data)
df

,review,label
0,"A câmera tem ótima qualidade, adorei!",1
1,"Bateria fraca, não dura nada.",0
2,"Design bonito e leve, mas o desempenho é ruim.",0
3,"Produto excelente, superou minhas expectativas.",1
4,"Não recomendo, veio com defeito.",0


#Classificação de Sentimento (BoW + SVM)

##Importações

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

##Vetorização

In [7]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["review"])
y = df["label"]

##Treinamento e avaliação

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


#Tópicos com LDA (Gensim)

##Importações

In [13]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim import corpora, models

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Pré-processamento

In [14]:
stop_words = set(stopwords.words('portuguese'))
texts = [[word.lower() for word in review.split() if word.lower() not in stop_words]
         for review in df["review"]]
stop_words

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estar',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houverí

### Geração do modelo

In [15]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = models.LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)

##Exibir tópicos

In [16]:
for idx, topic in lda.print_topics():
    print(f"Tópico {idx}: {topic}")

Tópico 0: 0.068*"superou" + 0.068*"expectativas." + 0.068*"excelente," + 0.068*"produto" + 0.068*"nada." + 0.068*"dura" + 0.068*"ótima" + 0.068*"bateria" + 0.068*"adorei!" + 0.068*"câmera"
Tópico 1: 0.083*"ruim." + 0.083*"desempenho" + 0.083*"bonito" + 0.083*"design" + 0.083*"leve," + 0.083*"defeito." + 0.083*"veio" + 0.083*"recomendo," + 0.028*"qualidade," + 0.028*"fraca,"
